In [17]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.exceptions import InvalidSignature
import time
import secrets
import hashlib

In [18]:
message = "The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks"

In [19]:
# Current time in seconds since the Unix epoch
timestamp = int(time.time())
print(f"Timestamp: {timestamp}")

Timestamp: 1708268105


In [20]:
# Generate a nonce as a hex string
nonce_hex = secrets.token_hex(16)  # Generates a 32-character hexadecimal string
print(f"Nonce (hex): {nonce_hex}")

Nonce (hex): b3e336849a28fbd799514427eccc2706


In [21]:
# Concatenate with a delimiter
combined = f"{nonce_hex}:{timestamp}:{message}"
print("Combined string:", combined)

Combined string: b3e336849a28fbd799514427eccc2706:1708268105:The company website has not limited the number of transactions a single user or device can perform in a given period of time. The transactions/time should be above the actual business requirement, but low enough to deter automated attacks


In [22]:
# Encode the combined string to bytes, then hash it using SHA-256
hashed = hashlib.sha256(combined.encode()).hexdigest()

print("SHA-256 Hash:", hashed)

SHA-256 Hash: e8005ebd0672593ac1306051fbb0f5298c502c1f10f0b2c66245b083147056fb


In [29]:
def generate_rsa_keys():
    """
    Generates an RSA private and public key pair.
    """
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

def serialize_rsa_keys(private_key, public_key):
    """
    Serializes the RSA private and public keys to PEM format.
    """
    pem_private_key = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()  # Use BestAvailableEncryption(password) for encrypted serialization
    )
    
    pem_public_key = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    return pem_private_key, pem_public_key

def sign_data_with_private_key(private_key, data_to_sign):
    """
    Signs data using the RSA private key.
    """
    signature = private_key.sign(
        data_to_sign,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature
